In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BSoup

In [2]:
url="https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168"
page=requests.get(url)
page

<Response [200]>

In [3]:
soup=BSoup(page.content,"html.parser")

In [8]:
seven_day=soup.find(id="seven-day-forecast")
forecast_items=seven_day.find_all(class_="forecast-tombstone")
tonight=forecast_items[0]
print(tonight.prettify())

<li class="forecast-tombstone">
 <div class="tombstone-container">
  <p class="period-name">
   Today
   <br/>
   <br/>
  </p>
  <p>
   <img alt="Today: Sunny, with a high near 66. Light and variable wind becoming west 8 to 13 mph in the afternoon. " class="forecast-icon" src="newimages/medium/skc.png" title="Today: Sunny, with a high near 66. Light and variable wind becoming west 8 to 13 mph in the afternoon. "/>
  </p>
  <p class="short-desc">
   Sunny
  </p>
  <p class="temp temp-high">
   High: 66 °F
  </p>
 </div>
</li>



In [9]:
period=tonight.find(class_="period-name").text
short_desc=tonight.find(class_="short-desc").text
temp=tonight.find(class_="temp").text
print(period)
print(short_desc)
print(temp)

Today
Sunny
High: 66 °F


In [10]:
img=tonight.find('img')
desc=img['title']
print(desc)

Today: Sunny, with a high near 66. Light and variable wind becoming west 8 to 13 mph in the afternoon. 


In [11]:
period_tags=seven_day.select(".tombstone-container .period-name")
periods=[pt.text for pt in period_tags]
print(periods)

['Today', 'Tonight', 'Sunday', 'SundayNight', 'Monday', 'MondayNight', 'Tuesday', 'TuesdayNight', 'Wednesday']


In [12]:
short_tags=seven_day.select(".tombstone-container .short-desc")
shorts=[st.text for st in short_tags]
print(shorts)

['Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Mostly Clear', 'Sunny']


In [13]:
temp_tags=seven_day.select(".tombstone-container .temp")
temps=[tt.text for tt in temp_tags]
print(temps)

['High: 66 °F', 'Low: 49 °F', 'High: 66 °F', 'Low: 49 °F', 'High: 66 °F', 'Low: 49 °F', 'High: 62 °F', 'Low: 47 °F', 'High: 61 °F']


In [14]:
img_tags=seven_day.select(".tombstone-container img")
descs=[d['title'] for d in img_tags]
print(descs)

['Today: Sunny, with a high near 66. Light and variable wind becoming west 8 to 13 mph in the afternoon. ', 'Tonight: Mostly clear, with a low around 49. North northwest wind 5 to 8 mph becoming calm. ', 'Sunday: Sunny, with a high near 66. East northeast wind 6 to 10 mph. ', 'Sunday Night: Mostly clear, with a low around 49. Calm wind becoming north northeast around 5 mph in the evening. ', 'Monday: Sunny, with a high near 66. Calm wind becoming north around 5 mph in the morning. ', 'Monday Night: Partly cloudy, with a low around 49.', 'Tuesday: Mostly sunny, with a high near 62.', 'Tuesday Night: Mostly clear, with a low around 47.', 'Wednesday: Sunny, with a high near 61.']


In [15]:
weather=pd.DataFrame({
    "period":periods,
    "short":shorts,
    "temp":temps,
    "desc":descs
})

In [16]:
weather

,period,short,temp,desc
0,Today,Sunny,High: 66 °F,"Today: Sunny, with a high near 66. Light and v..."
1,Tonight,Mostly Clear,Low: 49 °F,"Tonight: Mostly clear, with a low around 49. N..."
2,Sunday,Sunny,High: 66 °F,"Sunday: Sunny, with a high near 66. East north..."
3,SundayNight,Mostly Clear,Low: 49 °F,"Sunday Night: Mostly clear, with a low around ..."
4,Monday,Sunny,High: 66 °F,"Monday: Sunny, with a high near 66. Calm wind ..."
5,MondayNight,Partly Cloudy,Low: 49 °F,"Monday Night: Partly cloudy, with a low around..."
6,Tuesday,Mostly Sunny,High: 62 °F,"Tuesday: Mostly sunny, with a high near 62."
7,TuesdayNight,Mostly Clear,Low: 47 °F,"Tuesday Night: Mostly clear, with a low around..."
8,Wednesday,Sunny,High: 61 °F,"Wednesday: Sunny, with a high near 61."


In [20]:
temp_nums=weather["temp"].str.extract("(\d+)", expand=False)

In [22]:
weather["temp_num"]=temp_nums.astype('int')

In [24]:
weather["temp_num"].mean()

57.22222222222222

In [33]:
is_night=weather["temp"].str.contains("Low")
weather["is_night"]=is_night

In [34]:
weather[is_night]

,period,short,temp,desc,temp_num,is_night
1,Tonight,Mostly Clear,Low: 49 °F,"Tonight: Mostly clear, with a low around 49. N...",49,True
3,SundayNight,Mostly Clear,Low: 49 °F,"Sunday Night: Mostly clear, with a low around ...",49,True
5,MondayNight,Partly Cloudy,Low: 49 °F,"Monday Night: Partly cloudy, with a low around...",49,True
7,TuesdayNight,Mostly Clear,Low: 47 °F,"Tuesday Night: Mostly clear, with a low around...",47,True
